# Gelbenzeiten

In [3]:
import pandas as pd
import json
import re
from lxml import html
import http.client
import threading
import urllib

In [4]:
df_german_cities = pd.read_json('german_cities.json', orient='records')
df_german_cities['translators_count'] = 0
df_german_cities['error'] = False

In [13]:
def count_per_location(city):    
    conn = http.client.HTTPConnection("www.gelbeseiten.de")

    headers = {
        'cache-control': "no-cache",
        'postman-token': "1f7b2ff1-40c9-7b75-9075-dc8cf092cd1e"
        }

    conn.request("GET", "/uebersetzungen/{city}".format(city=urllib.parse.quote(city)), headers=headers)
    
    error = False

    try:
        res = conn.getresponse()
        data = res.read()
        body = data.decode("latin-1")

        tree = html.fromstring(body)
        elem = tree.xpath('//span[@class="gs_titel_anzahlTreffer"]/text()')
        count = int(elem[0])
    except Exception as e:
#         print(e)
#         print(body)
#         if 'elem' in locals():
#             print(elem)
        count = 0
        error = True
    
    return count, error

In [18]:
for idx, elm in df_german_cities[(df_german_cities.error==False) & (df_german_cities.translators_count==0)].iterrows():
    df_german_cities.loc[idx, ['translators_count', 'error']] = count_per_location(elm['city'].lower())
    print("\r{0:.2f}% Complete. {city}. {count} Translators. {errors} Errors.".format(
            100*((idx+1)/df_german_cities.shape[0]), 
            city=elm['city'], 
            count=df_german_cities['translators_count'].sum(),
            errors=df_german_cities['error'].sum()
        ), end="")

KeyboardInterrupt: 

In [16]:
df_german_cities[(df_german_cities.error==False) & (df_german_cities.translators_count==0)]

,city,translators_count,error
1354,Oldenburg,0,False
1355,Oldenburg in Holstein,0,False
1356,Olfen,0,False
1357,Olpe,0,False
1358,Olsberg,0,False
1359,Oppenau,0,False
1360,Oppenheim,0,False
1361,Oranienbaum-Wörlitz,0,False
1362,Oranienburg,0,False
1363,Orlamünde,0,False


In [17]:
df_german_cities.to_csv("german_cities_translators_count.json", orient="records")